# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
import os

print(os.uname())

# In the cloud environment?
if 'root' in os.environ['HOME']:
    UENVPATH = '/data/'
    !pip -q install /home/workspace/python

# In the standalone environment?
if 'ferenc' in os.environ['HOME']:
    UENVPATH = '/home/ferenc/Python/rl/udadrl/data/'


from unityagents import UnityEnvironment
import numpy as np

import pprint as pp

from collections import deque

from time import time

# Import the helper files
from utilities import get_time_string, print_elapsed_time

import pdb # Debug! Debug! Debug! Debug! Debug! Debug! 

posix.uname_result(sysname='Linux', nodename='Anubis-Linux', release='5.4.0-48-generic', version='#52-Ubuntu SMP Thu Sep 10 10:58:49 UTC 2020', machine='x86_64')


Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
print(get_time_string(sep=True), ' Notebook for Continuous Control started.')

# ONE Agent, Standalone
#UENVCHOICE = 'Reacher_Linux/Reacher.x86_64'

# TWENTY Agents, Standalone
#UENVCHOICE = 'Reacher_Linux_20/Reacher.x86_64'

# ONE Agent, Cloud, No-Visuals 
#UENVCHOICE = 'Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64'

# TWENTY Agents, Cloud, No-Visuals 
UENVCHOICE = 'Reacher_Linux_NoVis/Reacher.x86_64'

env = UnityEnvironment( file_name=os.path.join( UENVPATH, UENVCHOICE ) )




2020-09-26 / 21-29-16  Notebook for Continuous Control started.


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# FA DEBUG! DEBUG! DEBUG! 
type(env)

unityagents.environment.UnityEnvironment

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [5]:
# FA  DEBUG! DEBUG! DEBUG! 
print(env.brain_names)
print( type(brain) )
brain.brain_name

['ReacherBrain']
<class 'unityagents.brain.BrainParameters'>


'ReacherBrain'

In [6]:
# FA  DEBUG! DEBUG! DEBUG!
print(f'Observation Space: {brain.vector_observation_space_size} dimensions of {brain.vector_observation_space_type} type') 
print(f'Action Space: {brain.vector_action_space_size} dimensions of {brain.vector_observation_space_type} type')

Observation Space: 33 dimensions of continuous type
Action Space: 4 dimensions of continuous type


### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [7]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [8]:
# Should a run with random actions be perfomed first?
RANDOMRUN = False

# Some reminders about lambda functions
# Thx2: https://realpython.com/python-lambda/
# Thx2: https://thispointer.com/python-how-to-use-if-else-elif-in-lambda-functions/
# thx2: https://www.blog.pythonlibrary.org/2016/05/24/python-101-an-intro-to-benchmarking-your-code/
import timeit

env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)

epc = 0
print(f'We have {num_agents} active agent' + ( lambda x: 's' if x>1 else '' )(num_agents) )

start_time = time()
setup = "from unityagents import UnityEnvironment"

while RANDOMRUN:
    epc += 1
    #print(f'\r#{epc}', end = ' ')
    print(f'\r#{str(epc).rjust(6)}', end = ' ')
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    #pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug! 
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    #print( timeit.timeit("env_info = env.step(actions)[brain_name]", setup) )
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    rewards = env_info.rewards
    nprewards = np.array(rewards)
    scores += rewards                         # update the score (for each agent)
    if np.sum(nprewards) > 0:
        #print(f'\r#{epc} Rewards = {rewards}')
        maskagent = nprewards > 0
        agentbefore = False
        for (nagent, action) in enumerate(actions):
            if maskagent[nagent]:
                if agentbefore:
                    print('\r#' + '&'.rjust(6), end = ' ')
                print(' -> Agent {:0>2d} got reward {:+.5f} for action: {}'.format(nagent+1, rewards[nagent], action))
                agentbefore = True
                #pp.pprint(list(action))
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
        
end_time = time()

print_elapsed_time(start_time, end_time, "for choosing random actions")

print('#')
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

We have 20 active agents

***--> Total Elapsed Runtime: 00:00:00 for choosing random actions
#
Total score (averaged over agents) this episode: 0.0


### Re check running time (Random run)
This time with the leanest code possible

In [9]:
# Should another run with random actions be perfomed?
RANDOMRUN = False

env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
#env_info = env.reset(train_mode=True)[brain_name] # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)

epc = 0
print(f'We have {num_agents} active agent' + ( lambda x: 's' if x>1 else '' )(num_agents) )

start_time = time()

while RANDOMRUN:
    epc += 1
    print(f'\r#{str(epc).rjust(6)}', end = ' ')
    #pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug! 
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    dones = env_info.local_done                        # see if episode finished
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
        
end_time = time()

print_elapsed_time(start_time, end_time, "for choosing fast random actions")

print('#')
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

We have 20 active agents

***--> Total Elapsed Runtime: 00:00:00 for choosing fast random actions
#
Total score (averaged over agents) this episode: 0.0


In [10]:
# thx2: https://github.com/udacity/deep-reinforcement-learning/blob/master/python/unityagents/brain.py
env_info.vector_observations

array([[ 0.00000000e+00, -4.00000000e+00,  0.00000000e+00,
         1.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -4.37113883e-08,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -1.00000000e+01,
         0.00000000e+00,  1.00000000e+00, -0.00000000e+00,
        -0.00000000e+00, -4.37113883e-08,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  7.12805176e+00,
        -1.00000000e+00, -3.63192368e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  3.92812490e-02],
       [ 0.00000000e+00, -4.00000000e+00,  0.00000000e+00,
         1.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -4.37113883e-08,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -1.00000000e+01,
         0.00000000e+00,  1.00000000e+00, -0.00000000e+

### When finished, you can close the environment.
#### Just not for now because unit testing below

In [11]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

##### FA: One BIG MISUNDERSTANDING & STACKS OF TENSORS (23-08-2020) --> #FA; BMSoT:
It has cost me several days if not weeks to get behind the fact that the [A2C sample implementation of Miguel](https://github.com/mimoralea/gdrl/blob/master/notebooks/chapter_11/chapter-11.ipynb) is working with **stacks of tensors** instead of single tensors. Which was especially hard to find because the PyTorch code looks exactly the same for both.

In the end, when I thought about it, it makes sense and is a nifty feature of PyTorch. It is just not obvious to people like me, without in deep insights in the inner workings of PyTorch. 

Furthermore the authors of the PyTorch documentation seem not to make it too visible, I had to dig it out of one of the function definitions I use, however inderectly over the layer defintion for a2cnet:

https://pytorch.org/docs/0.4.0/_modules/torch/nn/functional.html#linear

> def linear(input, weight, bias=None):
>    """
>    Applies a linear transformation to the incoming data: :math:`y = xA^T + b`.
>
>    Shape:
>        - Input: :math:`(N, *, in\_features)` where `*` means any number of
>          additional dimensions
>        - Weight: :math:`(out\_features, in\_features)`
>        - Bias: :math:`(out\_features)`
>        - Output: :math:`(N, *, out\_features)`
>    """
>    if input.dim() == 2 and bias is not None:
>        # fused op is marginally faster
>        return torch.addmm(bias, input, weight.t())
>
>    output = input.matmul(weight.t())
>    if bias is not None:
>        output += bias
>    return output


#### Tracing the error in the 'mya2cnet' module code refactoring below

In [12]:
# TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! 
#import pdb; pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug! 

import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(20200808) # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!
#torch.manual_seed(456454618181) # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!

# Format: IN_Num [Layer 1] (OUT_Num = IN_Num) [Layer 2] OUT_Num = ...
HIDDEN_DIMS_DEFAULT = {
    'shared' : (512, 512, 256, 256),
    'actor' : (256, 128, 128, 64),
    'critic' : (256, 128, 128, 64)
}
hidden_dims = HIDDEN_DIMS_DEFAULT

hlayers = dict()

hlayers['shared'] = nn.ModuleList()
hlayers['actor'] = nn.ModuleList()
hlayers['critic'] = nn.ModuleList()




In [13]:
# Input layer
input_layer = nn.Linear( 33, hidden_dims['shared'][0] )

In [14]:
input_layer

Linear(in_features=33, out_features=512, bias=True)

In [15]:
# Hidden layers shared
for i in range( len(hidden_dims['shared']) -1 ):
    hlayers['shared'].append( nn.Linear( hidden_dims['shared'][i], hidden_dims['shared'][i+1] ) )

In [16]:
hlayers['shared']

ModuleList(
  (0): Linear(in_features=512, out_features=512, bias=True)
  (1): Linear(in_features=512, out_features=256, bias=True)
  (2): Linear(in_features=256, out_features=256, bias=True)
)

In [17]:
# Actor layers
for i in range( len(hidden_dims['actor']) ):
    #import pdb; pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!
    if i == 0:
        hlayers['actor'].append( nn.Linear( hidden_dims['shared'][-1], hidden_dims['actor'][i] ) )
    else:
        # hlayers['actor'].append( nn.Linear( hidden_dims['actor'][i], hidden_dims['actor'][i+1] ERROR !!!
        hlayers['actor'].append( nn.Linear( hidden_dims['actor'][i-1], hidden_dims['actor'][i] ) )
    #print( i, hlayers['actor'] ) # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!
        
actor_out_layer = nn.Linear( hidden_dims['actor'][-1], 4 )

In [18]:
hlayers['actor']

ModuleList(
  (0): Linear(in_features=256, out_features=256, bias=True)
  (1): Linear(in_features=256, out_features=128, bias=True)
  (2): Linear(in_features=128, out_features=128, bias=True)
  (3): Linear(in_features=128, out_features=64, bias=True)
)

In [19]:
actor_out_layer

Linear(in_features=64, out_features=4, bias=True)

In [20]:
#Critic layers
for i in range( len(hidden_dims['critic']) ):
    if i == 0:
        hlayers['critic'].append( nn.Linear( hidden_dims['shared'][-1], hidden_dims['critic'][i] ) )
    else:
        hlayers['critic'].append( nn.Linear( hidden_dims['critic'][i-1], hidden_dims['critic'][i] ) )
critic_out_layer = nn.Linear( hidden_dims['critic'][-1], 1 )

In [21]:
hlayers['critic']

ModuleList(
  (0): Linear(in_features=256, out_features=256, bias=True)
  (1): Linear(in_features=256, out_features=128, bias=True)
  (2): Linear(in_features=128, out_features=128, bias=True)
  (3): Linear(in_features=128, out_features=64, bias=True)
)

In [22]:
critic_out_layer

Linear(in_features=64, out_features=1, bias=True)

In [23]:
# Prevents non Pytorch Tensor Object entering the processing stream
def torch_format(state):
    x = state
    if not isinstance(x, torch.Tensor):
        x = torch.tensor(x, dtype=torch.float32)
        if len(x.size()) == 1:
            x = x.unsqueeze(0)
    return x

In [24]:
def forward(state):
    check_tensor = lambda x: isinstance(x, torch.Tensor)
    x_act = True 
    x_crit = True

    x = torch_format(state)
    x = F.relu(  input_layer(x) )
    for label in ['shared', 'actor', 'critic']:
        for hlayer in  hlayers[label]:
            if label == 'shared':
                x = F.relu(  hlayer(x) )
            if label == 'actor':
                x_act = F.relu(  hlayer(x_act) )
            if label == 'critic':
                x_crit = F.relu(  hlayer(x_crit) )

        # Thx2: https://discuss.pytorch.org/t/copy-deepcopy-vs-clone/55022
        if ( type(x_act) == bool ):
            x_act = x.clone()  # Create an Inplace copy...
        if ( type(x_crit) == bool ):
            x_crit = x.clone() # ...after processing shared layers

    return  actor_out_layer(x_act),  critic_out_layer(x_crit) 

#### Test if A STACK OF (BMSoT) states are propagated through the debug network
And make a list of outputs of two A2C instances

In [25]:
#al = []
#bl = []

#for (dc, st) in enumerate(states):
#    al.append(a)
#    bl.append(b)

#import pdb; pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!    
    
#FA; BMSoT: No need to iterate through states any more!
a,b = forward(states)
print(f'Actor: {a}')
print(f'Critic: {b}')

Actor: tensor(1.00000e-02 *
       [[ 4.6543,  4.9756, -4.4953,  4.2128],
        [ 4.7037,  4.9775, -4.4215,  4.1041],
        [ 4.7006,  4.9798, -4.4235,  4.1018],
        [ 4.6981,  4.9901, -4.3603,  4.1175],
        [ 4.7117,  4.9911, -4.4272,  4.1191],
        [ 4.6864,  4.9768, -4.4898,  4.2664],
        [ 4.7017,  5.0059, -4.4374,  4.2481],
        [ 4.7107,  4.9870, -4.4269,  4.1114],
        [ 4.6985,  4.9721, -4.4752,  4.2727],
        [ 4.6935,  5.0025, -4.4124,  4.2141],
        [ 4.7007,  4.9718, -4.3907,  4.1061],
        [ 4.6997,  4.9750, -4.4772,  4.2800],
        [ 4.6790,  4.9917, -4.4376,  4.1197],
        [ 4.6685,  4.9951, -4.4505,  4.1421],
        [ 4.6763,  4.9944, -4.4406,  4.1248],
        [ 4.6967,  4.9936, -4.3724,  4.1618],
        [ 4.7116,  4.9925, -4.4258,  4.1176],
        [ 4.7030,  4.9784, -4.4800,  4.2864],
        [ 4.7116,  4.9804, -4.4134,  4.1002],
        [ 4.6749,  4.9665, -4.4912,  4.2393]])
Critic: tensor(1.00000e-02 *
       [[ 4.7572],
   

#### Test if A STACK OF (BMSoT) states is propagated through the imported network

In [26]:
import pdb # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!

#from mya2cnet import A2CNetwork
import mya2cnet

# Thx2: https://emacs.stackexchange.com/a/13483
import imp
imp.reload(mya2cnet)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

tstnet1 = mya2cnet.A2CNetwork(33, 4).to(device) #FA; BMSoT: Looks exactly the same...
tstnet2 = mya2cnet.A2CNetwork(33, 4).to(device) #FA; BMSoT: ...like without stacks!

In [27]:
#FA; BMSoT: OBSOLETE
#for (dc, st) in enumerate(states):



# pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!
    
#a1,b1 = tstnet1.forward(torch.tensor(states, dtype=torch.float, device=device))
#a2,b2 = tstnet2.forward(torch.tensor(states).to(device))

#print(f'Dist. Actor stacks 1-2: {torch.dist(a1, a2)}'.ljust(50) + ' # ' +\
#     f'Dist. Actor 1 stacks - Notebook stacks {torch.dist(a1, a)}'.rjust(50))
#print(f'Dist. Critic stacks 1-2: {torch.dist(b1, b2)}'.ljust(50) + ' # ' +\
#     f'Dist. Critic 1 stacks - Notebook stacks {torch.dist(b1, b)}'.rjust(50))

In [28]:
#next(tstnet1.parameters()).is_cuda

In [29]:
#next(tstnet2.parameters()).is_cuda

In [30]:
#FA; BMSoT: OBSOLETE
#for (dc, st) in enumerate(states):
#    print(f'({dc+1}) -> {tstnet1.fullpass(st)}')
    
#tstnet1.fullpass( torch.tensor(states, device = device, dtype = torch.float) )

In [31]:
#FA; BMSoT: OBSOLETE
#for (dc, st) in enumerate(states):
#    print(f'({dc+1})-> {tstnet1.select_action(st)}') #, end=' ')
    
#tstnet1.select_action(states)

#### Test the agent instance

In [32]:
import mya2cagent

# Thx2: https://emacs.stackexchange.com/a/13483
import imp
imp.reload(mya2cagent)

agent = mya2cagent.a2cagent(len(env_info.agents), env, brain, max_steps = 500, max_n_steps = 5)

In [33]:
# Thx2: https://discuss.pytorch.org/t/how-to-check-if-model-is-on-cuda/180/2?u=ferenc_acs
start_time = time()

agent.train()

end_time = time()

print_elapsed_time(start_time, end_time, "for training the agent")

 --> Actions:
mean: 0.01808287083606173 sdev: 0.07060895642852447
Training iteration: 1            last optimization: 0+ --> Actions:
mean: 0.01807734779361262 sdev: 0.07063281014675264
Training iteration: 2            last optimization: 0# --> Actions:
mean: 0.018071174317049327 sdev: 0.07065008669578025
Training iteration: 3            last optimization: 0+ --> Actions:
mean: 0.01806348189600602 sdev: 0.07066046072974423
Training iteration: 4            last optimization: 0# --> Actions:
mean: 0.01805669696022093 sdev: 0.07066249225850192
Training iteration: 5            last optimization: 5+ --> Actions:
mean: 0.02401661100221554 sdev: 0.05739678617372673
Training iteration: 6            last optimization: 5# --> Actions:
mean: 0.024007067233461597 sdev: 0.057397730060849535
Training iteration: 7            last optimization: 5+ --> Actions:
mean: 0.023994723792416184 sdev: 0.05739900105979525
Training iteration: 8            last optimization: 5# --> Actions:
mean: 0.02398220817050

In [34]:
# Some reminders about lambda functions
# Thx2: https://realpython.com/python-lambda/
# Thx2: https://thispointer.com/python-how-to-use-if-else-elif-in-lambda-functions/

RANDOMRUN = False


env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
#env_info = env.reset(train_mode=True)[brain_name] # Debug! Debug! Debug! Debug! Debug! Debug! 
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
avg100sum = np.zeros(num_agents)
exectime = 0

scores100 = deque(avg100sum, 100)
time100 = list()
epc = 0


#import pdb; pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug! 
start_time = time()

while RANDOMRUN: #== True:
    epc += 1
    #print(f'\r#{epc}', end = ' ')
    print(f'\r#{str(epc).rjust(6)}', end = ' ')
    #actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = agent.a2c_net.select_action(states)
    #actions = np.clip(actions.detach().cpu().numpy(), -1, 1)                  # all actions between -1 and 1
        
    t_step_b = time()
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    t_step_e = time()
    time100.append(t_step_e - t_step_b)
    
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    rewards = env_info.rewards
    nprewards = np.array(rewards)
    scores += rewards                         # update the score (for each agent)
    scores100.append(rewards)
    if np.sum(nprewards) > 0:
        #for x in scores100:
        #    avg100sum += x
        print(f'\r#{epc} Avg 100 Rewards = {np.mean(scores100)}')  
        avg100sum = np.zeros(num_agents)
        maskagent = nprewards > 0
        
    if epc%100 == 0:
        print(f'Exec time Avg 100: {np.mean(time100)}')
        time100 = []
        
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
        
end_time = time()

print_elapsed_time(start_time, end_time, "for observing the trained agent")
        
print('#')
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))


***--> Total Elapsed Runtime: 00:00:00 for observing the trained agent
#
Total score (averaged over agents) this episode: 0.0


In [35]:
env.close()

In [36]:
print(get_time_string(sep=True), ' Notebook for Continuous Control ended.')

2020-09-26 / 21-29-23  Notebook for Continuous Control ended.
